In [7]:
import torch
print(torch.cuda.is_available())

True


### Yolo Large model

In [19]:
import cv2
from ultralytics import YOLO
from collections import defaultdict

# Load the YOLO model
model = YOLO('yolo11l.pt')

In [27]:
class_list = model.names 
#class_list
print(class_list)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [1]:
import cv2
from ultralytics import YOLO
import torch
from collections import defaultdict

# ✅ Load YOLO model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO('yolo11m.pt').to(device)

# ✅ Open video file
cap = cv2.VideoCapture('4.mp4')

# ✅ Get frame height to position red line at center
ret, test_frame = cap.read()
if not ret:
    print("Error: Cannot read video file.")
    cap.release()
    exit()

frame_height = test_frame.shape[0]
line_y_red = frame_height // 2  # red line in center

# ✅ Vehicle classes (car, motorcycle, bus, truck)
vehicle_classes = [2, 3, 5, 7]

# ✅ Counters and tracking
class_counts = defaultdict(int)
crossed_ids = set()
frame_id = 0
frame_skip = 2
total_count = 0  # total vehicles counted

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    if frame_id % frame_skip != 0:
        continue

    # Resize for better performance
    frame = cv2.resize(frame, (640, 360))
    line_y_red = frame.shape[0] // 2  # recalc for resized frame

    # ✅ Run YOLO tracking
    results = model.track(frame, persist=True, classes=vehicle_classes, conf=0.6, verbose=False)

    # ✅ Draw red line
    cv2.line(frame, (50, line_y_red), (590, line_y_red), (0, 0, 255), 3)
    cv2.putText(frame, "Counting Line", (50, line_y_red - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # ✅ Process detections
    if results and results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu()
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        for box, cls_idx, tid in zip(boxes, class_indices, track_ids):
            x1, y1, x2, y2 = map(int, box)
            cx, cy = (x1 + x2)//2, (y1 + y2)//2
            class_name = model.names[cls_idx]

            # Draw detection info
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.circle(frame, (cx, cy), 3, (0, 0, 255), -1)
            cv2.putText(frame, f"{class_name} ID:{tid}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

            # ✅ Count only once when crossing the line
            if cy > line_y_red and tid not in crossed_ids:
                crossed_ids.add(tid)
                class_counts[class_name] += 1
                total_count += 1
                print(f" Vehicle crossed! Total count: {total_count}")

                # ✅ Simulate traffic light event
                if total_count == 20:
                    print("\n⚠️ 20 vehicles crossed! Traffic light changing sequence begins...")
                    print("🟡 Yellow Light - Prepare to Stop...")
                    cv2.putText(frame, "Yellow Light - Prepare to Stop", (100, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 3)
                    cv2.imshow("YOLO Vehicle Detection & Counting", frame)
                    cv2.waitKey(2000)

                    print("🔴 Red Light - Stop for 5 seconds...\n")
                    cv2.putText(frame, "Red Light - STOP", (180, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
                    cv2.imshow("YOLO Vehicle Detection & Counting", frame)
                    cv2.waitKey(5000)

                    print("🟢 Green Light - Resuming Traffic!\n")
                    total_count = 0  # Reset after light cycle
                    crossed_ids.clear()  # reset IDs after cycle

    # ✅ Display count info
    y = 30
    cv2.putText(frame, "Vehicle Counts:", (20, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    y += 25
    for cls, count in class_counts.items():
        cv2.putText(frame, f"{cls}: {count}", (20, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        y += 25

    cv2.imshow("YOLO Vehicle Detection & Counting", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

 Vehicle crossed! Total count: 1
 Vehicle crossed! Total count: 2
 Vehicle crossed! Total count: 3
 Vehicle crossed! Total count: 4
 Vehicle crossed! Total count: 5
 Vehicle crossed! Total count: 6
 Vehicle crossed! Total count: 7
 Vehicle crossed! Total count: 8
 Vehicle crossed! Total count: 9
 Vehicle crossed! Total count: 10
 Vehicle crossed! Total count: 11
 Vehicle crossed! Total count: 12
 Vehicle crossed! Total count: 13
 Vehicle crossed! Total count: 14
 Vehicle crossed! Total count: 15
 Vehicle crossed! Total count: 16
 Vehicle crossed! Total count: 17
 Vehicle crossed! Total count: 18
 Vehicle crossed! Total count: 19
 Vehicle crossed! Total count: 20

⚠️ 20 vehicles crossed! Traffic light changing sequence begins...
🟡 Yellow Light - Prepare to Stop...
🔴 Red Light - Stop for 5 seconds...

🟢 Green Light - Resuming Traffic!

 Vehicle crossed! Total count: 1
 Vehicle crossed! Total count: 2
 Vehicle crossed! Total count: 3
 Vehicle crossed! Total count: 4
 Vehicle crossed! Tota

In [3]:
import cv2
from ultralytics import YOLO
import torch
from collections import defaultdict

# ✅ Load YOLO model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO('yolo11m.pt').to(device)

# ✅ Open video file
cap = cv2.VideoCapture('4.mp4')

# Line position (a bit higher)
line_y_red = 260  

# ✅ Vehicle classes
vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck

# ✅ Counters and tracking memory
class_counts = defaultdict(int)
crossed_ids = set()  # remember which IDs have crossed

# ✅ Frame skip to improve speed
frame_id = 0
frame_skip = 2  

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    if frame_id % frame_skip != 0:
        continue

    frame = cv2.resize(frame, (640, 360))

    # ✅ Run YOLO tracking (not just detect)
    results = model.track(frame, persist=True, classes=vehicle_classes, conf=0.6, verbose=False)

    # ✅ Draw counting line
    cv2.line(frame, (100, line_y_red), (540, line_y_red), (0, 0, 255), 3)
    cv2.putText(frame, "Counting Line", (100, line_y_red - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # ✅ Process tracked detections
    if results and results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu()
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        for box, cls_idx, tid in zip(boxes, class_indices, track_ids):
            x1, y1, x2, y2 = map(int, box)
            cx, cy = (x1 + x2)//2, (y1 + y2)//2
            class_name = model.names[cls_idx]

            # Draw tracking box & label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.circle(frame, (cx, cy), 3, (0, 0, 255), -1)
            cv2.putText(frame, f"ID:{tid} {class_name}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

            # ✅ Count vehicle only ONCE when it crosses the line
            if cy > line_y_red and tid not in crossed_ids:
                crossed_ids.add(tid)
                class_counts[class_name] += 1

    # ✅ Display count info
    y = 30
    cv2.putText(frame, "Vehicle Counts:", (20, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    y += 25
    for cls, count in class_counts.items():
        cv2.putText(frame, f"{cls}: {count}", (20, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        y += 25

    cv2.imshow("YOLO Vehicle Detection & Counting", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### Faster Execution

In [21]:
# Open the video file
cap = cv2.VideoCapture('4.mp4')

In [29]:
import cv2
from ultralytics import YOLO
from collections import defaultdict

# Load the YOLO model
model = YOLO('yolo11l.pt')

class_list = model.names 
#class_list

# Open the video file
cap = cv2.VideoCapture('4.mp4')

line_y_red = 430  # Red line position

# Dictionary to store object counts by class
class_counts = defaultdict(int)

# Dictionary to keep track of object IDs that have crossed the line
crossed_ids = set()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run YOLO tracking on the frame
    results = model.track(frame, persist=True, classes = [1,2,3,5,6,7]) 
    #print(results)

    # Ensure results are not empty
    if results[0].boxes.data is not None:
        # Get the detected boxes, their class indices, and track IDs
        boxes = results[0].boxes.xyxy.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        class_indices = results[0].boxes.cls.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu()

        cv2.line(frame, (690, line_y_red), (1130, line_y_red), (0, 0, 255), 3)
        #cv2.putText(frame, 'Red Line', (690, line_y_red - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)


        

        # Loop through each detected object
        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2  # Calculate the center point
            cy = (y1 + y2) // 2            

            class_name = class_list[class_idx]

            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            
            cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) 


            # Check if the object has crossed the red line
            if cy > line_y_red and track_id not in crossed_ids:
                # Mark the object as crossed
                crossed_ids.add(track_id)
                class_counts[class_name] += 1


        # Display the counts on the frame
        y_offset = 30
        for class_name, count in class_counts.items():
            cv2.putText(frame, f"{class_name}: {count}", (50, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            y_offset += 30

    
    
    # Show the frame
    cv2.imshow("YOLO Object Tracking & Counting", frame)    
    
    # Exit loop if 'q' key is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


0: 384x640 10 cars, 3 motorcycles, 3 buss, 1 truck, 47.6ms
Speed: 3.0ms preprocess, 47.6ms inference, 16.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 3 motorcycles, 3 buss, 1 truck, 127.2ms
Speed: 4.2ms preprocess, 127.2ms inference, 21.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 3 motorcycles, 3 buss, 1 truck, 312.5ms
Speed: 3.8ms preprocess, 312.5ms inference, 42.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 3 motorcycles, 3 buss, 1 truck, 47.3ms
Speed: 1.6ms preprocess, 47.3ms inference, 23.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 4 motorcycles, 3 buss, 1 truck, 47.0ms
Speed: 2.1ms preprocess, 47.0ms inference, 24.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 4 motorcycles, 3 buss, 1 truck, 48.0ms
Speed: 3.0ms preprocess, 48.0ms inference, 21.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 4 motorcycles, 3 buss, 2 tru

### Yolo medium model

In [31]:
import cv2
from ultralytics import YOLO
from collections import defaultdict
import torch

model = YOLO('yolo11m.pt').to('cuda' if torch.cuda.is_available() else 'cpu')

cap = cv2.VideoCapture('Sample.mp4')
class_counts = defaultdict(int)
crossed_ids = set()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640, 360))  # reduce resolution for speed
    height, width, _ = frame.shape
    line_y_red = int(height * 0.6)  # draw line at 75% of frame height

    results = model.track(frame, persist=True, classes=[1,2,3,5,6,7])
    if results and results[0].boxes.data is not None:
        boxes = results[0].boxes.xyxy.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        class_indices = results[0].boxes.cls.int().cpu().tolist()

        # Red line
        cv2.line(frame, (int(width * 0.2), line_y_red), (int(width * 0.85), line_y_red), (0, 0, 255), 3)

        for box, tid, cls_idx in zip(boxes, track_ids, class_indices):
            x1, y1, x2, y2 = map(int, box)
            cx, cy = (x1 + x2)//2, (y1 + y2)//2
            class_name = model.names[cls_idx]

            cv2.circle(frame, (cx, cy), 3, (0, 0, 255), -1)
            cv2.putText(frame, f"{class_name}", (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)

            if cy > line_y_red and tid not in crossed_ids:
                crossed_ids.add(tid)
                class_counts[class_name] += 1

        y = 30
        for cls, count in class_counts.items():
            cv2.putText(frame, f"{cls}: {count}", (30, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
            y += 30

    cv2.imshow("YOLO Tracking", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 7 cars, 1 truck, 38.8ms
Speed: 2.6ms preprocess, 38.8ms inference, 7.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 117.5ms
Speed: 2.9ms preprocess, 117.5ms inference, 18.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 269.2ms
Speed: 2.4ms preprocess, 269.2ms inference, 21.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 205.2ms
Speed: 2.7ms preprocess, 205.2ms inference, 18.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 36.9ms
Speed: 3.0ms preprocess, 36.9ms inference, 17.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 36.1ms
Speed: 1.4ms preprocess, 36.1ms inference, 13.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 36.4ms
Speed: 1.7ms preprocess, 36.4ms inference, 17.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 truck, 36.1ms
Speed: 2.0ms preproc

### faster execution